# Introduction à l'exploration intéractive de données avec Python et Pandas

Auteur: Valentin Barriere

Objectif: Comprendre le contenu des données, la structure, les problèmes (données manquantes / aberrantes)

Inspiré des travaux d'Alexandre Gramfort

### Les données:

MovieLens 1M Data Set contient les notes attribuées à des films par des utilisateurs du site Movielens. Les données sont fournies, mais peuvent être trouvées, si besoin, à l'adresse: http://grouplens.org/datasets/movielens/

### Chargement de packages utiles

In [2]:
import pandas as pd  #pour l'exploration de données
import numpy as np   #pour les operations numeriques (type Matlab)

### Lire les données "users" dans un DataFrame Pandas

In [3]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv('users.dat', sep='::', header=None, names=unames, engine='python')

In [4]:
users.head() # head() permet de n'afficher que les premieres lignes pour la visualisation

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


### Lire les données "rating" dans un DataFrame Pandas

In [5]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ratings.dat', sep='::', header=None, names=rnames, engine='python')

In [6]:
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


### Lire les données sur les films  dans un  Data Frame Pandas

In [7]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('movies.dat', sep='::', header=None, names=mnames, engine='python')

In [8]:
movies.head(10)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


### Fusion des données en un seul DataFrame

In [9]:
data = pd.merge(pd.merge(ratings, users), movies)

In [10]:
data.head() #head() permet de n'afficher que les premieres lignes pour la visualisation

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [11]:
len(data)

1000209

# Explorons !

### Question 0 : Les bases 

Combien de notes as-t-on dans la base de données ? Est-ce une majorité d'homme ou de femme ? Quel est l'age moyen et la variance ? 

Ecrire une fonction qui print la somme et la moyenne d'un `pandas.Series` de type `bool`

In [12]:
len(data["rating"])
data.groupby(by="gender", as_index=False).count()
homme= data["gender"]=="M"
femme=data["gender"]=="F"

In [13]:
def sum_and_mean(dbool):
    """
    Fonction pour savoir le nombre de booléen 
    """
    somme=dbool.sum()
    moyenne=dbool.mean()*100
    print ('Somme : %d ; Pourcentage : %.2f'%(somme, moyenne))

In [14]:
hommes=sum_and_mean(homme)
femmes=sum_and_mean(femme)

Somme : 753769 ; Pourcentage : 75.36
Somme : 246440 ; Pourcentage : 24.64


### Question 1 : Conditions sur les booléens

Combien de films ont une note supérieure à 4.5 ? Existe-t-il une différence entre les hommes et les femmes?

In [15]:
sup_4_5=len(data.loc[data["rating"]>4.5])
m_rating=len(data.loc[(data["rating"]>4.5) & (data["gender"]=="M")])
f_rating=len(data.loc[(data["rating"]>4.5) & (data["gender"]=="F")])

m_rating/f_rating            


2.8655074642161718

Attention il vaut mieux regarder les proportions pour avoir une réponse pertinente:

In [16]:
m_rating/len(data.loc[data["gender"]=="M"])
f_rating/len(data.loc[data["gender"]=="F"])

0.23756695341665315

On verifie que c'est bien la même chose 

In [17]:
 sup_4_5/len(data)
data.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp', 'gender', 'age',
       'occupation', 'zip', 'title', 'genres'],
      dtype='object')

### Question 2 : Operations booléennes et groupement

Combien de films ont une note médiane au dessus de 4,5 parmis les hommes de plus de 30 ans? parmis les femmes de plus de 30 ans?

In [18]:
h_30_plus=data.loc[(data["gender"]=="M") & (data["age"]> 30)]
f_30_plus=data.loc[(data["gender"]=="F") & (data["age"]> 30)]


median_h=h_30_plus.groupby(by="movie_id", as_index=False)["rating"].median()

median_f=f_30_plus.groupby(by="movie_id", as_index=False)["rating"].median()

#Combien de films ont une note médiane au dessus de 4,5 parmis les hommes de plus de 30 ans
print(len(median_h.loc[median_h["rating"]>4.5]))

#parmis les femmes de plus de 30 ans?
print(len(median_f.loc[median_f["rating"]>4.5]))



86
149


### Question 3 : Faire attention à la taille des données

Quels sont les films les mieux notés ?

In [19]:
#300 films les mieux notés
best=data.groupby(by="title",as_index=False)["rating"].mean().sort_values(by="rating",ascending=False).head(300)
print(best)



                                                  title    rating
3477                            Ulysses (Ulisse) (1954)  5.000000
2025                                       Lured (1947)  5.000000
1203                            Follow the Bitch (1998)  5.000000
407                            Bittersweet Motel (2000)  5.000000
3087                             Song of Freedom (1936)  5.000000
2453                           One Little Indian (1973)  5.000000
3044                               Smashing Time (1967)  5.000000
2903          Schlafes Bruder (Brother of Sleep) (1995)  5.000000
1297                 Gate of Heavenly Peace, The (1995)  5.000000
249                                    Baby, The (1973)  5.000000
1622                I Am Cuba (Soy Cuba/Ya Kuba) (1964)  4.800000
1870                                    Lamerica (1994)  4.750000
199                             Apple, The (Sib) (1998)  4.666667
2883                                     Sanjuro (1962)  4.608696
2940  Seve

Est-ce que ce sont vraiment les plus populaires ? 

In [117]:
#300 films les plus populaires
top_films=data.groupby(by="title",as_index=False).count().sort_values(by="user_id",ascending=False).loc[:,['title',"user_id"]].head(300)

#On récupère tous les avis de chaque film faisant parti du top 300
top_films_detail=data.merge(top_films,on="title")

#fims les plus populaires groupés par note moyenne
top_films_rating=top_films_detail.groupby(by="title",as_index=False)["rating"].mean().sort_values(by="rating",ascending=False)

#On ajoute une colonne rang pour la note
top_films_rating["rang_rating"]=top_films_rating["rating"].rank(ascending=False)

#On crée un df qui a le count de chaque film
top_films_count=top_films_detail.groupby(by="title",as_index=False).count().sort_values(by="gender",ascending=False).loc[:,['title',"user_id_x"]].head(300)

#On renomme la colonne user_id_x en count
top_films_count.columns=["title","count"]

#On ajoute une colonne rang pour le count

top_films_count["rang_count"]=top_films_count["count"].rank(ascending=False)


#on fusionne les deux top films

top_films_final= top_films_rating.merge(top_films_count, on="title")

print(top_films_final.iloc[:,1:])

       rating  rang_rating  count  rang_count
0    4.554558          1.0   2227        24.0
1    4.524966          2.0   2223        25.0
2    4.517106          3.0   1783        39.0
3    4.510417          4.0   2304        18.0
4    4.507937          5.0    882       269.0
5    4.477725          6.0   2514        12.0
6    4.476190          7.0   1050       187.0
7    4.453694          8.0   2991         2.0
8    4.449890          9.0   1367        91.5
9    4.425647         10.0    928       240.5
10   4.412822         11.0   1669        51.0
11   4.406263         12.0   2459        14.0
12   4.401925         13.0    831       295.0
13   4.395973         14.0   1043       192.0
14   4.390725         15.0   1725        44.0
15   4.388889         16.0   1116       160.5
16   4.386994         17.0    938       235.5
17   4.384030         18.0   1315       106.5
18   4.357565         19.0   1692        50.0
19   4.351823         20.0   2578        10.0
20   4.339241         21.0   1185 

On va definir une popularité minimale (en fonction du nombre de notes obtenues), et garder uniquement les films qui sont au dessus d'un tel seuil. 

In [121]:
# seuil= 1000

populaire_1000=top_films_final.loc[top_films_final["count"]>=1000]

Quel est  le film le plus souvent noté par les utilisateurs? 

In [136]:
populaire_1000.loc[populaire_1000["count"].idxmax()]

title          American Beauty (1999)
rating                        4.31739
rang_rating                        26
count                            3428
rang_count                          1
Name: 25, dtype: object